In [1]:
!pip install 'influxdb-client[ciso]'
!pip install --upgrade firebase-admin
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import csv
import os
import pandas as pd
import random
import time
import sys
import datetime
from influxdb_client import InfluxDBClient, Point, Dialect
from datetime import date
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Influx DB

client = InfluxDBClient.from_config_file("config.ini") 
query_api = client.query_api() 

#Firebase
default_app = firebase_admin.initialize_app()

In [3]:
def month_date(month_no):
    test_date = datetime.datetime(2021, month_no, 1)
    nxt_mnth = test_date.replace(day=28) + datetime.timedelta(days=4)
    res = nxt_mnth - datetime.timedelta(days=nxt_mnth.day)
    arr=[test_date,res]
    return arr

def monthly_consumption(month_no):
    spr = month_date(month_no)
    if month_no == 10:
        spr[0] += timedelta(days=12)
        stri= '''from(bucket: "sme-akash")|> range(start: {start}, stop: {stop})|> filter(fn: (r) => r["_field"] == "energy")|> spread()|> yield(name: "spread")'''.format(start=str(spr[0].strftime("%s")), stop=str(spr[1].strftime("%s")))
        sp = query_api.query_data_frame(stri)
        sp.at[1,'_value'] += 33
        return sp
    else:
        stri= '''from(bucket: "sme-akash")|> range(start: {start}, stop: {stop})|> filter(fn: (r) => r["_field"] == "energy")|> spread()|> yield(name: "spread")'''.format(start=str(spr[0].strftime("%s")), stop=str(spr[1].strftime("%s")))
        sp = query_api.query_data_frame(stri)
        return(sp)


In [4]:
db = firestore.client()
def calc(units,solar=0,kwh=9):
    net=units-solar
    fixed=85+(kwh-1)*95
    amt=0
    if net<=50: amt=net * 4.1
    elif net<=100: amt=205+(net-50)*5.55
    elif net<=200: amt=205+277.5+(net-100)*7.1
    else: amt=205+277.5+710+(net-200)*8.15
    amt=amt*1.09-50-14.22
    return amt+fixed

def returnResult(date, solar_val, current_month_consumption, remaining_units, current_estimated_consumption):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month_str = months[date.month-1]
    print("You have consumed", round(current_month_consumption, 2), "units this month, and you have", round(remaining_units, 2) ,"units left!")
    print("Total consumption estimate:", round(current_estimated_consumption, 2), "units and solar generation:", round(solar_val, 2), "units.")
    print("Estimated bill amount for", month_str, ":", "Rs.", round(calc(current_estimated_consumption, solar_val), 2))
    
    today = date.today()

    doc_ref = db.collection(u'predictions').document(u'month_end_bill')
    doc_ref.set({
    u'current_month_consumption': round(current_month_consumption, 2),
    u'remaining_units': round(remaining_units, 2),
    u'current_estimated_consumption': round(current_estimated_consumption, 2),
    u'solar_val': round(solar_val, 2),
    u'estimated_bill': round(calc(current_estimated_consumption, solar_val), 2),
    u'last_updated': today
    })
    
    print("wrote data to firebase")
    
    print(type(today))

def predict(date):
    month = date.month

    prev_month_1 = monthly_consumption(month - 1)
    prev_month_1_solar = prev_month_1.loc[prev_month_1["_measurement"]=="Solar"]
    prev_month_1_phases = prev_month_1.loc[prev_month_1["_measurement"]!="Solar"]

    prev_month_1_solar_generation = prev_month_1_solar["_value"].sum()
    prev_month_1_consumption = prev_month_1_phases["_value"].sum()


    prev_month_2 = monthly_consumption(month - 2)
    prev_month_2_solar = prev_month_2.loc[prev_month_2["_measurement"]=="Solar"]
    prev_month_2_phases = prev_month_2.loc[prev_month_2["_measurement"]!="Solar"]

    prev_month_2_solar_generation = prev_month_2_solar["_value"].sum()
    prev_month_2_consumption = prev_month_2_phases["_value"].sum()

    current_month = monthly_consumption(month)
    current_month_phases = current_month.loc[current_month["_measurement"]!="Solar"]
    current_month_consumption = current_month_phases["_value"].sum()

    predicted_solar = 0.5*(prev_month_2_solar_generation + prev_month_1_solar_generation)

    if(prev_month_1_consumption > prev_month_2_consumption):
      current_estimated_consumption = 2*prev_month_1_consumption - prev_month_2_consumption
    else: 
      current_estimated_consumption = 0.5*(prev_month_1_consumption+prev_month_2_consumption)

    remaining_units = current_estimated_consumption - current_month_consumption
    returnResult(date, predicted_solar, current_month_consumption, remaining_units, current_estimated_consumption)

date = pd.Timestamp('2021-10-25 08:00:29.745000').tz_localize(None)
predict(date)

You have consumed 220.18 units this month, and you have 299.4 units left!
Total consumption estimate: 519.58 units and solar generation: 276.73 units.
Estimated bill amount for October : Rs. 2461.31
wrote data to firebase
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
